In [53]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)
import lightgbm as lgb
from sklearn.metrics import log_loss

In [2]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [118]:
df.columns

Index(['NAME', 'Rowid', 'XY水道水位置情報', 'X', 'Y', 'up_down', 'curvature',
       'inclination', 'tilt_direction', 'altitude', 'disto_river',
       'disto_stations', 'disto_mainroad', 'disto_syorizyo', 'supply_hours',
       'no_water_days', 'total_population ', 'population_served',
       'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe',
       'served/pipes', '(popu-served)/pipes', 'oldest_pipe_age', 'ST', 'RSF',
       'FL', 'PF', 'RF', 'EColi.'],
      dtype='object')

In [119]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [120]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [121]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [122]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [123]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

# random forest

In [124]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(max_depth=20, criterion='entropy', n_estimators=40, random_state=1)
forest.fit(X_train_std, y_train)

RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1)

In [125]:
forest.score(X_train_std, y_train)

1.0

In [126]:
forest.score(X_test_std, y_test)

0.7727272727272727

# light gbm

In [47]:
X_train_std

,up_down,curvature,inclination,tilt_direction,altitude,disto_river,disto_stations,disto_mainroad,disto_syorizyo,supply_hours,no_water_days,total_population,population_served,popu-served,number_taps,pipelength,pipelength_per_pipe,served/pipes,(popu-served)/pipes,oldest_pipe_age,ST,RSF,FL,PF,RF
218,-0.686850,-0.382187,0.048990,-0.410915,-1.159545,-0.327503,-1.265976,-0.135508,-0.070461,0.939600,-1.478259,-1.032852,-1.008378,-1.129822,-1.022905,-1.036154,-0.031214,-1.256239,-0.508817,-0.784953,1,1,0,0,0
100,1.831442,-1.612959,0.077027,1.317255,0.879267,0.004160,0.949064,-0.448003,-0.178807,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
147,-0.686850,0.110122,0.083950,0.323822,0.939576,0.527216,1.045501,-0.147035,-0.094643,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
190,-0.327094,0.725508,0.080769,0.610908,-1.188024,-0.724693,-0.980053,-0.163948,-0.394109,0.380087,-0.877575,-1.024947,-1.014043,-1.062279,-1.033523,-1.070890,-0.023260,-1.165195,0.956916,-1.091538,0,0,0,0,0
192,-0.226362,-0.012955,0.082693,-1.455383,-1.146142,-0.883740,-1.183121,1.309977,-0.016418,-0.515133,0.040044,-0.925869,-0.999170,-0.593647,-1.008196,-1.059415,-0.179687,-0.272058,5.788956,-1.295928,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,-0.686850,-0.751419,0.072918,0.057667,-0.975264,-0.532585,-0.258298,0.944393,0.031416,1.611015,-1.406907,-0.858066,-0.925323,-0.553148,-0.861831,-0.523634,-0.042145,-1.324718,-0.331289,-0.963794,0,0,0,0,0
156,-0.686850,1.463971,0.085376,-0.390698,0.931200,-0.800241,1.049782,-0.202599,0.026095,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
16,-0.686850,0.233199,0.080769,-1.263421,-1.132740,-0.364624,-1.329180,-0.221402,-0.399950,0.044380,-0.334968,-1.026417,-1.014292,-1.069108,-1.031457,-1.066652,-0.043024,-1.269909,0.629604,-0.963794,0,0,0,0,0
144,-0.686850,-0.136033,0.084819,0.003578,1.019990,0.350658,1.092903,-0.304388,0.119831,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1


In [93]:
# 特徴量と目的変数をlightgbmのデータ構造に変換する
lgb_train = lgb.Dataset(X_train_std, y_train)
lgb_eval = lgb.Dataset(X_test_std, y_test)

# ハイパーパラメータの設定
params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}
num_round = 1500

# 学習の実行
# カテゴリ変数をパラメータで指定している
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
categorical_features = ['ST', 'RSF','FL', 'PF', 'RF']
model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  categorical_feature=categorical_features,
                  valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])

# バリデーションデータでのスコアの確認
train_prob=model.predict(X_train_std)
test_prob = model.predict(X_test_std)
train_los= log_loss(y_train, train_prob)
test_los= log_loss(y_test, test_prob)
print(f'train_logloss: {train_los:.4f}')
print(f'test_logloss: {test_los:.4f}')

# accuracy
def bunrui(a):
    if a<0.5:
        return 0
    else:
        return 1
    
train_pred=pd.DataFrame({'class':train_prob})['class'].apply(bunrui)
test_pred=pd.DataFrame({'class':test_prob})['class'].apply(bunrui)

train_accuracy = sum(y_train.values == train_pred.values) / len(y_train)
test_accuracy = sum(y_test.values == test_pred.values) / len(y_test)
print(train_accuracy)
print(test_accuracy)

C:\Users\kurokiso\anaconda3\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['FL', 'PF', 'RF', 'RSF', 'ST']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\kurokiso\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\kurokiso\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's binary_logloss: 0.666925	valid's binary_logloss: 0.676053
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's binary_logloss: 0.648346	valid's binary_logloss: 0.671509
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's binary_logloss: 0.632476	valid's binary_logloss: 0.671311
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's binary_logloss: 0.615951	valid's binary_logloss: 0.666656
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's binary_l

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[105]	train's binary_logloss: 0.219725	valid's binary_logloss: 0.687769
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[106]	train's binary_logloss: 0.218398	valid's binary_logloss: 0.688658
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[107]	train's binary_logloss: 0.217396	valid's binary_logloss: 0.690439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[108]	train's binary_logloss: 0.215914	valid's binary_logloss: 0.692457
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[109]	train's binary_logloss: 0.213627	valid's binary_logloss: 0.692273
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[110]	train's binary_logloss: 0.211992	valid's binary_logloss: 0.692957
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[111]	train's binary_logloss: 0.210362	vali

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[218]	train's binary_logloss: 0.0925175	valid's binary_logloss: 0.852252
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[219]	train's binary_logloss: 0.0917653	valid's binary_logloss: 0.852969
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[220]	train's binary_logloss: 0.090918	valid's binary_logloss: 0.85625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[221]	train's binary_logloss: 0.090208	valid's binary_logloss: 0.857656
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[222]	train's binary_logloss: 0.0895817	valid's binary_logloss: 0.859104
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[223]	train's binary_logloss: 0.0889319	valid's binary_logloss: 0.861395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[224]	train's binary_logloss: 0.088461	v

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[343]	train's binary_logloss: 0.0373691	valid's binary_logloss: 0.975779
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[344]	train's binary_logloss: 0.0372	valid's binary_logloss: 0.976627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[345]	train's binary_logloss: 0.0370152	valid's binary_logloss: 0.979141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[346]	train's binary_logloss: 0.0367207	valid's binary_logloss: 0.980772
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[347]	train's binary_logloss: 0.0364353	valid's binary_logloss: 0.985245
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[348]	train's binary_logloss: 0.0362357	valid's binary_logloss: 0.983309
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[349]	train's binary_logloss: 0.0358867	

...

[1040]	train's binary_logloss: 0.000298757	valid's binary_logloss: 1.84542
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1041]	train's binary_logloss: 0.000296592	valid's binary_logloss: 1.84703
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1042]	train's binary_logloss: 0.000295089	valid's binary_logloss: 1.8466
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1043]	train's binary_logloss: 0.00029296	valid's binary_logloss: 1.85016
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1044]	train's binary_logloss: 0.000290915	valid's binary_logloss: 1.85337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1045]	train's binary_logloss: 0.000288304	valid's binary_logloss: 1.85639
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1046]	train's binary_logloss: 0.000286637	valid's binary_logloss: 1.85767
[LightGBM] [Warning] No fur

[1153]	train's binary_logloss: 0.00013749	valid's binary_logloss: 1.99537
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1154]	train's binary_logloss: 0.000136795	valid's binary_logloss: 1.99701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1155]	train's binary_logloss: 0.000135396	valid's binary_logloss: 1.99844
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1156]	train's binary_logloss: 0.000134559	valid's binary_logloss: 2.00159
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1157]	train's binary_logloss: 0.000133209	valid's binary_logloss: 1.99946
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1158]	train's binary_logloss: 0.000132425	valid's binary_logloss: 2.00353
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1159]	train's binary_logloss: 0.000131491	valid's binary_logloss: 2.00279
[LightGBM] [Warning] No fu

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1268]	train's binary_logloss: 6.46243e-05	valid's binary_logloss: 2.10485
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1269]	train's binary_logloss: 6.41999e-05	valid's binary_logloss: 2.1085
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1270]	train's binary_logloss: 6.37512e-05	valid's binary_logloss: 2.11086
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1271]	train's binary_logloss: 6.30343e-05	valid's binary_logloss: 2.11465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1272]	train's binary_logloss: 6.27108e-05	valid's binary_logloss: 2.11524
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1273]	train's binary_logloss: 6.2497e-05	valid's binary_logloss: 2.11573
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1274]	train's binary_loglo

[1393]	train's binary_logloss: 3.71891e-05	valid's binary_logloss: 2.19697
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1394]	train's binary_logloss: 3.71157e-05	valid's binary_logloss: 2.19595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1395]	train's binary_logloss: 3.69742e-05	valid's binary_logloss: 2.19447
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1396]	train's binary_logloss: 3.68794e-05	valid's binary_logloss: 2.1957
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1397]	train's binary_logloss: 3.67886e-05	valid's binary_logloss: 2.1957
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1398]	train's binary_logloss: 3.67117e-05	valid's binary_logloss: 2.19534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1399]	train's binary_logloss: 3.6628e-05	valid's binary_logloss: 2.19586
[LightGBM] [Warning] No furt

train_logloss: 0.0000
test_logloss: 2.2200
1.0
0.696969696969697
